In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import seaborn as sns
%matplotlib inline

# Векторизация текста

<p>Для многих задач связанные с текстом нам бы хотелось представлять текст в виде вектора, чтобы похожие слова или предложения располагались рядом с друг другом. Для чего это нужно?. Например для задач поиска, нам хотелось бы выдавать ответ так, чтобы наиболее похожие статьи находились в верху списка. Такой тип представления текстов и слов называется дистрибутивная семантика.</p> 
<p>Каким же образом мы можем представить слова в таком виде?</p>

## PMI

<p>Логичным методом представления слова является, контекст в котором оно употребляются в тексте</p>
<p>Контекстом мы будем считать слова, которые находятся рядом с искомым словом в окне определенного размера, например для слова <i>лес</i> и окна 3, контекст выглядит следующим образом</p>
<p>Житель <b>местности, покрытой хвойным</b> <i>лесом</i>, <b>связывает с «деревом»</b> прежде всего образ ели или сосны.</p>
<p>Исходя из этой идеи, мы можем посчитать вектор слова следующим образом</p>

- Посчитать слова которые встречаются с данным словом в одном контексте
- Создать вектор где в определенное значение записана частота определенного слова из словаря

<p>В таком случае похожесть между словами мы можем измерять с помощью косинусного расстояния между их векторами</p>
<p>Однако такой способ имеет большой недостаток, а именно часто встречаемые слова (например предлоги) будут иметь очень большой вес. Чтобы уменьшить влияние таких слов, вместо частоты встречаемости слов, будем считать PMI (pointwise mutual information)</p>
<p><center>$$\large{PMI = \log{\frac{p(u, v)}{p(v)p(u)}}=\log{\frac{n_{uv}n}{n_un_v}}}$$</center></p>
<p>Однако из с данным способом представления встречаемости слов есть проблема, для редко встречаемых между собой слов получается большое отрицательно значение, а если слова никогда не встречаются между собой то значение равно $-\infty$. Поэтому на практике пользуются pPMI</p>
<p><center>$$\large{pPMI = \max{(0, PMI)}}$$</center></p>

## Truncated SVD

<p>И так у нас имеется способ представление совстречаемости слов с помощью pPMI. Так же мы можем мерить похожесть слов между собой с помощью косинусного расстояния. Однако у нас есть другая проблема а именно, размерность вектора каждого слова равна длинне словаря и затраты на вычисление похожести слов слишком большие. Поэтому воспользуюмся техникой снижения размерности а именно Truncated SVD</p>
<p>Truncated SVD представляет нашу матрицу размера (W*W) в виде произведения двух матриц (W*K) и (K*W), где K - новая размерность для наших векторов</p>
<img src="./img/pmi_svd.png">
<p>Для того что найти наилучшие матрицы для такого произведения мы решаем задачу минимизации расстояния между исходной матрицей и скалярного произведения новых матриц</p>
<p><center>$\large{||X-X_f||^2 \rightarrow min}$</center></p>

## GloVe

<p>GloVe - Global Vectors for Word Representation алгоритм представления слов в виде векторов, был разработан в университет Стендфорд. Почитать о нем можно по <a href="https://nlp.stanford.edu/projects/glove/">ссылке</a>, <a href="https://github.com/stanfordnlp/GloVe">git</a> проекта где можно скачать вектора</p>
<p>GloVe похож на вектора которые получаются с помощью SVD из pPMI матрицы, однако во время поиска матрицы представления слов, минимизируется другая ошибка и матрица заполнена логорифомом встречаемости слов:</p>
<p><center>$$\large{\sum_{u \in W}\sum_{v \in W}f(n_{uv})(\phi_u\theta_v + b_u + b_v - \log{n_{uv}})^2}$$</center></p>
<p>Это делается для того чтобы уменьшить влияние часто встречаемых слов</p>

## Skip-gram

<p>В данном случае мы пытаемся построить модель, которая бы предсказывала контекст по заданному слову:</p>
<p><center>$$\large{p(\omega_{i-k}..\omega_{i+k}|\omega_{i}) = \prod_{-k\le j\le k, j\ne 0}p(\omega_{i+j}|\omega{i})}$$</center></p>
<p>Где вероятность встречаемости каждого слова, представлена как:</p>
<p><center>$$\large{p(u|v) = \frac{\exp{\phi_u\theta_v}}{\sum_{\dot{u}\in W}\exp{\phi_\dot{u}\theta_v}}}$$</center></p>
<p>Так же как в версии с Truncated SVD у нас есть матрицы $\Phi$ и $\Theta$, в которых находятся векторы слов, в skip-gram мы максимизируем логарифмическое правдоподобие, вместо минимизации MSE</p>
<p><center>$$\large{L = \sum_{u \in W}\sum_{v \in W}n_{uv}\log{p(u|v)}}$$</center></p>
<p>Однако на практике данной моделью пользуются редко, поскольку ее обучение занимает очень много времени из-за того что нам нужно считать софтмакс по всему словарю</p>

## Skip-gram Negative Sampling

<p>Данная модель абсолютно аналогична Skip-gram за исключением того что функция максимизации состоит из 2 частей:</p>

- для положительных примеров (слов из контекста) 
<p><center>$$\large{L_{positive} = \sum_{u \in W}\sum_{v \in W}n_{uv}\log{\sigma(\phi_u\theta_v)}}$$</center></p>

- для отрицательных примеров (слов вне контекста), выбираем случайные k слов 
<p><center>$$\large{L_{negative} = kE_v\log{\sigma(-\phi_u\theta_v)}}$$</center></p>

<p>Полная метрика выглядит как сумма позитивной и негативной метрик</p>
<p><center>$$\large L = L_{positive}+L_{negative}$$</center></p>
<p>Интересным фактом является то что оптимальное значение для скалярного произведения skip-gram векторов является смещенная оценка PMI</p>
<p><center>$$\large{sPMI = PMI - \log{k}}$$</center></p>

## CBOW

<p>Архитектура CBOW аналогична Skip-gram, с одним отличием, мы пытаемся максимизировать вероятность найти слово по заданному контексту</p>

## Doc2Vec

<p>Doc2Vec это набор моделей которые к векторам слов, добавляют вектор документов для этих слов, сделанно это для того чтобы:</p>

- Получить векторное представление документа
- Использовать информацию о документе для того чтобы различать омонимы

<p>В Doc2Vec представленно 2 модели</p>

- Distributed memory version of Paragraph Vector

<p><center>$$\large{p(\omega_{i}|\omega_{i-k}..\omega_{i+k}, d)}$$</center></p>

- Distributed Bag of Words version of Paragraph Vector

<p><center>$$\large{p(\omega_{i-k}..\omega_{i+k}|d)}$$</center></p>

<p>Skip-Gram, CBOW, Doc2Vec представлены в пакете <a href="https://radimrehurek.com/gensim/">gensim</a></p>

## FastText

<p>Данная модель похожа на Skip-gram Negative Sampling, только вместо скалярного произведения вектора слов, мы используем скалярное произведение для сумм char-gram</p>
<p>Пример 3char-gram для слова <i>семантика</i>: __с, _се, сем, ема, ман, ант, нти, тик, ика</p>
<p><center>$$\large{\log{\sigma(\phi_u\theta_v)} = \log{\sigma(\sum_{c \in word}\phi_u c_v})}$$</center></p>
<p><a href="https://github.com/facebookresearch/fastText/">git FastText</a></p>

## StarSpace

<p>В реальных задачах нам чаще нужны вектора целых предложений, вместо векторов слов нам нужны вектора целых предложений, существует несколько подходов для их получения</p>

- среднее среди всех векторов в тексте
- взвешенная сумма по idf векторов в тексте
- пакет StarSpace

<p>Пакет StarSpace можно найти на <a href="https://github.com/facebookresearch/StarSpace">git</a>, идея данной модели очень схожа с идеей FastText только вместо похожести слов, мы ищем похожие предложения, каждое из которых представленно в виде сумм векторов слов. В результате мы получаем оптимизированные вектора слов для представления предложений</p>

In [2]:
vector_file = open('data/cc.ru.300.vec', 'r')

In [3]:
vector_file.seek(0)

0

In [4]:
vector_file.readline()

'2000000 300\n'

In [5]:
words_dict = {}

In [6]:
vectors = []

In [7]:
for i, line in enumerate(vector_file):
    line = line.rstrip().split(' ')
    words_dict[line[0]] = i
    vectors.append(np.array([float(var) for var in line[1:]]).astype(np.float16))

In [8]:
vectors = np.array(vectors)

In [24]:
j = np.argsort(vectors.dot(vectors[words_dict['кролик']]))[-5:]

In [25]:
[k for k,v in words_dict.items() if v in j]

['пёс', 'бык', 'еж', 'ёж', 'Ёж']

In [29]:
j = np.argsort(vectors.dot(vectors[words_dict['программирование']]))[-5:]

In [30]:
[k for k,v in words_dict.items() if v in j]

['PHP', '1с', 'Qt', 'dq', 'UML']